In [5]:
from dataInput import *
from CV import *
from FA import *
import numpy as np
import pandas as pd

In [2]:
inputParams = inputParams(1,1,57,1000);

In [3]:
intensityFile = 'data/synthetic/intensity.csv';
locationFile = 'data/synthetic/location.csv';
landfallFile = 'data/synthetic/landfall_7.csv';

In [4]:
hurricaneDataSet = hurricaneInput(intensityFile, locationFile, landfallFile, inputParams)

nodeLists =  [[56], [1, 2, 3, 8, 9, 10, 15, 16, 17, 22, 23, 24, 29, 30, 31, 36, 37, 38, 43, 44, 45, 50, 51, 52, 57, 58, 59, 64, 65, 66, 71, 72, 73, 78, 79, 80, 85, 86, 87, 92, 93, 94, 99, 100, 101, 106, 107, 108, 113, 114, 115, 120, 121, 122, 127, 128, 129, 134, 135, 136, 141, 142, 143], [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31, 32, 33, 34, 37, 38, 39, 40, 41, 44, 45, 46, 47, 48, 51, 52, 53, 54, 55, 58, 59, 60, 61, 62, 65, 66, 67, 68, 69, 72, 73, 74, 75, 76, 79, 80, 81, 82, 83, 86, 87, 88, 89, 90, 93, 94, 95, 96, 97, 100, 101, 102, 103, 104, 107, 108, 109, 110, 111, 114, 115, 116, 117, 118, 121, 122, 123, 124, 125, 128, 129, 130, 131, 132, 135, 136, 137, 138, 139, 142, 143, 144, 145, 146, 149, 150, 151, 152, 153, 156, 157, 158, 159, 160, 163, 164, 165, 166, 167, 170, 171, 172, 173, 174, 177, 178, 179, 180, 181, 184, 185, 186, 187, 188, 191, 192, 193, 194, 195], [3, 4, 5, 6, 10, 11, 12, 13, 17, 18, 19, 20, 24, 25, 26, 27, 31, 32, 33, 34, 38, 39, 

In [ ]:
netNodesFile = 'data/synthetic/nodes.csv';
netParamsFile = 'data/synthetic/netParams.csv';
networkDataSet = networkInput(3,30,0.05,netNodesFile,netParamsFile,hurricaneDataSet)

In [ ]:
osfname = f"data/synthetic/OOS57.csv"
OS_paths = pd.read_csv(osfname).values  # Read the out-of-sample file

In [ ]:
results = clairvoyant_eval(networkDataSet,hurricaneDataSet,inputParams)